## QCLab: Simon’s Algorithm – 2‑Qubit Proof of Concept

Simon’s algorithm, shown in the figure, finds a hidden period $s \in \{0,1\}^n$ (with $s \neq 0$) such that a function $f:\{0,1\}^n \to \{0,1\}^n$ satisfies  $f(x) = f(y) \;\;\text{if and only if}\;\; y = x \oplus s .$
In other words, the function outputs the same value for $x$ and for $x \oplus s$, and the task of Simon’s algorithm is to determine the period $s$.

![Simons-algorithm](images/Simons-algorithm.png)

The quantum circuit creates a superposition, queries the oracle, and applies Hadamards to extract information about $s$.  
Classical post‑processing solves:  $y \cdot s \equiv 0 \ (\text{mod}\ 2)$ to recover $s$.

---

### Task
- Design and run a minimal implementation of Simon’s algorithm with two input qubits and fixed $s = 10$.
- The oracle should ensure $s = 10$ is the only valid hidden string.  
- Run until $n - 1 = 1$ independent equation is collected, which is enough to recover $s$ for $n = 2$.

### Expected Output
- Display the created Simon’s quantum circuit  
- Display collected equations $y \cdot s \equiv 0 \ (\text{mod}\ 2)$  
- Recovered hidden string matches $s$  

### Experimentation
Extend to $n=3$ qubits, allow user‑defined $s$, compare runs for different $n$, and observe measurement distributions.


In [ ]:
# ====================================================
# QCLab: Simon’s Algorithm
# <QC|CT> qcict.org
# ====================================================

from IPython.display import display

from qiskit import QuantumCircuit, QuantumRegister,ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import numpy as np


def simon_oracle():
    """
    Simon oracle for n = 2, s = 10
    Mapping:
        00 -> 00
        10 -> 00
        01 -> 10
        11 -> 10
    """
    x = QuantumRegister(2, name='x')
    f = QuantumRegister(2, name='f')
    qc = QuantumCircuit(x, f, name="Oracle")

    # Output[0] = x0
    qc.cx(x[0], f[0])

    # Output[1] always 0 (no gates needed)

    return qc

def build_simon_circuit(oracle):
    """
    Build a Simon's algorithm circuit:
    H on inputs → oracle → H on inputs → measure.
    """
    n = oracle.num_qubits // 2
    x = QuantumRegister(n, name='x')
    f = QuantumRegister(n, name='f')
    c = ClassicalRegister(n, name='c')
    qc = QuantumCircuit(x, f, c)

    # First Hadamards
    qc.h(x)
    qc.barrier()

    # Oracle
    qc.compose(oracle, qubits=x[:] + f[:], inplace=True)

    qc.barrier()

    # Second Hadamards
    qc.h(x)

    # Measurement
    qc.measure(x, c)

    # Optional: Draw circuit for visualization
    display(circuit_drawer(qc, style="bw", output="mpl"))

    return qc

def find_all_solutions(equations, n):
    """
    Classical step: solve y · s = 0 (mod 2) for all measurement equations.
    Returns all non-zero bitstrings s that satisfy the system.
    """
    solutions = []
    
    # Try all possible bitstrings except the all-zero string
    for guess in range(1, 2**n):
        # Convert the integer to a zero-padded binary string
        guess_str = format(guess, f'0{n}b')
        
        # Convert the string to a list of integers for math
        guess_bits = list(map(int, guess_str))
        
        # Check if this guess satisfies all equations y · s = 0 (mod 2)
        valid = True
        for eq in equations:
            dot_product = sum(a * b for a, b in zip(eq, guess_bits)) % 2
            if dot_product != 0:
                valid = False
                break
        
        # If it satisfies all equations, add it to the list
        if valid:
            solutions.append(guess_str)
    
    return solutions

def run_simon_algorithm(shots=1000):
    """
    Runs Simon's algorithm for a fixed oracle until enough independent equations are collected. 
    Executes the circuit repeatedly, measures inputs, and stores unique equations y · s = 0 (mod 2).  
    Stops when collected equations reach rank n − 1, ensuring the hidden string is found.  
    Returns the chosen hidden string, the list of equations, and the measurement counts.
    """
    n = 2
    equations = []
    oracle = simon_oracle()
    qc = build_simon_circuit(oracle)
    simulator = AerSimulator()

    # Repeat until enough independent equations (rank n−1) are collected
    while np.linalg.matrix_rank(np.array(equations)) < n - 1:
        result = simulator.run(qc, shots=shots).result()
        counts = result.get_counts(qc)

        # Process each measured bitstring and add it if it increases rank
        for bitstring in counts:
            row = [int(b) for b in bitstring]
            if row not in equations:
                if np.linalg.matrix_rank(np.array(equations + [row])) > np.linalg.matrix_rank(np.array(equations)):
                    equations.append(row)
            if np.linalg.matrix_rank(np.array(equations)) == n - 1:
                break

    all_solutions = find_all_solutions(equations, n)
    chosen_s = all_solutions[0] if all_solutions else None
    return chosen_s, equations

# ------------------------------------------------
#                main program
# ------------------------------------------------

found_s, eqs = run_simon_algorithm(shots=100)

print("Equations collected (y · s = 0):", eqs)
print("Chosen hidden string:", found_s)
print("Actual hidden string: 10")
